In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [4]:
spark = SparkSession.builder\
    .appName('de-course')\
    .master('local[*]')\
    .getOrCreate()

22/03/02 22:16:15 WARN Utils: Your hostname, HGDEWLFR001442 resolves to a loopback address: 127.0.1.1; using 172.30.172.238 instead (on interface eth0)
22/03/02 22:16:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 22:16:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Question 1
spark.version

'3.0.3'

In [5]:
# Question 2 download data
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 19:57:10--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.102.155
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.102.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  5.78MB/s    in 5m 35s  

2022-03-01 20:02:46 (2.09 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [6]:
input_path='fhvhv_tripdata_2021-02.csv'

In [18]:
# add schema
schema = types.StructType([
types.StructField('hvfhs_license_num',types.StringType(),True),
types.StructField('dispatching_base_num',types.StringType(),True),
types.StructField('pickup_datetime',types.TimestampType(),True),
types.StructField('dropoff_datetime',types.TimestampType(),True),
types.StructField('PULocationID',types.IntegerType(),True),
types.StructField('DOLocationID',types.IntegerType(),True),
types.StructField('SR_Flag',types.IntegerType(),True)
])

In [19]:
#read parquet with schema
df_hvfhw = spark.read.csv(input_path, inferSchema=True, header=True, schema=schema)

In [25]:
df_hvfhw.repartition(24).write.parquet('pq/hvfhw')

22/03/01 20:24:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 20:24:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/03/01 20:24:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
22/03/01 20:24:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/03/01 20:24:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
22/03/01 20:24:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/03/01 20:24:45 WARN MemoryManager: Total allocation exceeds 95.

In [3]:
!du -sh pq/hvfhw

205M	pq/hvfhw


In [7]:
df_hvfhw = spark.read.parquet('pq/hvfhw')

In [8]:
df_hvfhw.registerTempTable('tripdata')

In [9]:
# Question 3. Count records
# How many taxi trips were there on February 15?
spark.sql("""
    SELECT count(*) FROM tripdata
    where date_trunc('DAY',pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [11]:
df_hvfhw.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02866|2021-02-02 11:23:35|2021-02-02 11:44:17|          10|          95|   null|
|           HV0003|              B02617|2021-02-01 14:06:08|2021-02-01 14:12:07|          85|          89|   null|
|           HV0003|              B02876|2021-02-03 22:11:13|2021-02-03 22:19:47|          90|         230|   null|
|           HV0003|              B02884|2021-02-02 18:29:33|2021-02-02 18:44:33|         160|         258|   null|
|           HV0003|              B02878|2021-02-01 20:03:14|2021-02-01 20:28:32|          39|          39|   null|
+-----------------+--------------------+-------------------+-------------------+

In [12]:
# Question 4. Longest trip for each day
# Now calculate the duration for each trip.
# Trip starting on which day was the longest?
spark.sql("""
    SELECT
        date_trunc('DAY',pickup_datetime) as day,
        unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime) as duration
    FROM tripdata
    ORDER BY 2 DESC
    LIMIT 4
""").show()

+-------------------+--------+
|                day|duration|
+-------------------+--------+
|2021-02-11 00:00:00|   75540|
|2021-02-17 00:00:00|   57221|
|2021-02-20 00:00:00|   44039|
|2021-02-03 00:00:00|   40653|
+-------------------+--------+



In [13]:
# Question 5. Most frequent dispatching_base_num
# Now find the most frequently occurring dispatching_base_num in this dataset.

spark.sql("""
    SELECT
        dispatching_base_num,
        count(1) as count
    FROM tripdata
    GROUP BY dispatching_base_num
    ORDER BY 2 DESC
    LIMIT 4
""").show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
+--------------------+-------+



In [14]:
df_zones = spark.read.parquet('pq/zones')
df_zones.registerTempTable('zones')

In [16]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [19]:
# Question 6. Most common locations pair
spark.sql("""
    SELECT
        zpu.Zone as pickup_zone,
        zdo.Zone as dropoff_zone,
        count(1) as count
    FROM tripdata t
    LEFT JOIN zones zpu on t.PULocationID = zpu.LocationID
    LEFT JOIN zones zdo on t.DOLocationID = zdo.LocationID
    GROUP BY 1,2
    ORDER BY 3 DESC
    LIMIT 4
""").show()

+-------------------+-------------------+-----+
|        pickup_zone|       dropoff_zone|count|
+-------------------+-------------------+-----+
|      East New York|      East New York|45041|
|       Borough Park|       Borough Park|37329|
|           Canarsie|           Canarsie|28026|
|Crown Heights North|Crown Heights North|25976|
+-------------------+-------------------+-----+

